In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
#("sqlite:///Resources/hawaii.sqlite")
session = Session(engine)

In [9]:
conn = engine.connect()
station_df = pd.read_sql('select * from station', conn)

station_df

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [10]:
measurement_df = pd.read_sql('select * from measurement', conn)
measurement_df

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0
...,...,...,...,...,...
19545,19546,USC00516128,2017-08-19,0.09,71.0
19546,19547,USC00516128,2017-08-20,NaN,78.0
19547,19548,USC00516128,2017-08-21,0.56,76.0
19548,19549,USC00516128,2017-08-22,0.50,76.0


In [11]:
merge_df = pd.merge(station_df,measurement_df, on='id', how='outer')

merge_df.head()

,id,station_x,name,latitude,longitude,elevation,station_y,date,prcp,tobs
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0,USC00519397,2010-01-01,0.08,65.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6,USC00519397,2010-01-02,0.00,63.0
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0,USC00519397,2010-01-03,0.00,74.0
3,4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9,USC00519397,2010-01-04,0.00,76.0
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6,USC00519397,2010-01-06,NaN,73.0


# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
value = np.nanmax(merge_df.date)

#print(maxValues)
print(value)


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
results = session.query(measurement.date, measurement.prcp).filter(measurement.date >= date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = pd.DataFrame(results, columns=['date', 'precipitation'])
precip_df.count()


In [ ]:
# Sort the dataframe by date
date_precip = precip_df.sort_values(by='date',ascending=True).dropna()

date_precip.head()

In [ ]:
# Plot Precipitation levels by year
date_precip.plot(x='date', color='steelblue', alpha=0.8, figsize=(9,5))
plt.xlabel("Date")
plt.ylabel("Inches")
plt.title("Precipitation Analysis (1 YR)")
plt.xticks(rotation='60')
#plt.legend(["Precipitation"])
plt.savefig("Images_final/Precipitation.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
date_precip.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(station).count()
print(station_count)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_active = session.query(measurement.station, func.count(measurement.station)).\
    group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()

station_active

In [ ]:
active_station = station_active[0]
active_station

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
filter(measurement.station == 'USC00519281').all()

In [ ]:
# plot hist
tempt_twelve = pd.DataFrame(session.query(measurement.tobs).filter(measurement.station == 'USC00519281').filter(measurement.date >= '2016-08-23').all(), columns=['tempt'])
tempt_twelve.plot.hist(bins=12)
plt.xlabel('Temperature')
plt.savefig('Images_final/Precipitation.png')

# Close session

In [ ]:
# Close Session
session.close()